# Naive Bayes — Student Lab

Implement Multinomial Naive Bayes from scratch (log-space + Laplace smoothing).

In [2]:
import re
import numpy as np

def check(name: str, cond: bool):
    if not cond:
        raise AssertionError(f'Failed: {name}')
    print(f'OK: {name}')

rng = np.random.default_rng(0)

## Section 0 — Toy dataset
We use a tiny spam/ham dataset so you can verify logic easily.

In [3]:
docs = [
    ('spam', 'win money now'),
    ('spam', 'free money win'),
    ('spam', 'claim your free prize'),
    ('ham',  'meeting schedule tomorrow'),
    ('ham',  'let us schedule a meeting'),
    ('ham',  'project update tomorrow'),
]

labels = np.array([1 if y=='spam' else 0 for y,_ in docs])
texts = [t for _,t in docs]

## Section 1 — Tokenization + Vocabulary

### Task 1.1: Tokenize

# TODO: implement tokenizer
# HINT: lowercase + split on non-letters

**FAANG gotcha:** punctuation/case can break counts.

In [4]:
def tokenize(text):
    # TODO
    tokens = re.split(r'[^a-zA-Z]+', text.lower())
    return [t for t in tokens if t]
    # return tokens

print(tokenize('Free MONEY!!!'))

['free', 'money']


### Task 1.2: Build vocabulary

# TODO: build vocab dict word->index from all docs

**Checkpoint:** What happens if a word appears only in test?

**Answer:** If a word appears only in the test set, it’s out-of-vocabulary and ignored, since the model can’t assign learned statistics to unseen words.

In [5]:
# TODO
vocab = {}
for t in texts:
  for w in tokenize(t):
    if w not in vocab:
      vocab[w] = len(vocab)

inv_vocab = {i:w for w,i in vocab.items()}
print("inverese vocabulary : ", inv_vocab)

print(vocab)
check('vocab_nonempty', len(vocab) > 0)

inverese vocabulary :  {0: 'win', 1: 'money', 2: 'now', 3: 'free', 4: 'claim', 5: 'your', 6: 'prize', 7: 'meeting', 8: 'schedule', 9: 'tomorrow', 10: 'let', 11: 'us', 12: 'a', 13: 'project', 14: 'update'}
{'win': 0, 'money': 1, 'now': 2, 'free': 3, 'claim': 4, 'your': 5, 'prize': 6, 'meeting': 7, 'schedule': 8, 'tomorrow': 9, 'let': 10, 'us': 11, 'a': 12, 'project': 13, 'update': 14}
OK: vocab_nonempty


## Section 2 — Vectorization (Count vectors)

### Task 2.1: Convert docs to count matrix X (n_docs, |V|)

# TODO
# HINT: loop over tokens inside a doc is ok; avoid loops over vocab per doc


In [6]:
def vectorize(texts, vocab):
    # TODO
    X = np.zeros((len(texts), len(vocab)), dtype=np.int64)
    for i, text in enumerate(texts):
      for w in tokenize(text):
        if w in vocab:
          X[i, vocab[w]] += 1
    return X

X = vectorize(texts, vocab)
print('X shape', X.shape)
check('shape', X.shape[0] == len(texts) and X.shape[1] == len(vocab))

X shape (6, 15)
OK: shape


## Section 3 — Train Multinomial Naive Bayes

### Task 3.1: Fit with Laplace smoothing

Compute:
- class priors P(y)
- word likelihoods P(word|y) with Laplace smoothing alpha

# HINT:
- count words per class: sum rows where y==c
- add alpha to each word count

**Checkpoint:** Why smoothing is required?

**Answer:** Smoothing prevents zero probabilities for unseen words, which would otherwise dominate the Naive Bayes score and make the classifier unusable.


In [7]:
def fit_nb(X, y, alpha=1.0):
    # TODO
    # return log_priors (2,), log_likelihoods (2, V)
    y = y.astype(int)
    V = X.shape[1]
    log_priors = np.zeros(2)
    log_lik = np.zeros((2, V))
    for c in (0, 1):
      Xc = X[y == c]
      log_priors[c] = np.log(Xc.shape[0] / X.shape[0])
      word_counts = Xc.sum(axis=0)
      smoothed = word_counts + alpha
      denom = smoothed.sum()
      log_lik[c] = np.log(smoothed / denom)
    return log_priors, log_lik

log_priors, log_lik = fit_nb(X, labels, alpha=1.0)
check('priors_shape', log_priors.shape == (2,))
check('lik_shape', log_lik.shape[0] == 2 and log_lik.shape[1] == X.shape[1])

OK: priors_shape
OK: lik_shape


### Task 3.2: Predict in log-space

For each doc:
log P(y=c|x) ∝ log P(y=c) + sum_j x_j * log P(word_j|c)

**Interview Angle:** Why use logs?

**Answer:** We use logs for numerical stability (avoid underflow) and faster computation (sums instead of products).

In [9]:
def predict_nb(X, log_priors, log_lik):
    # TODO
    scores = X @ log_lik.T + log_priors  # broadcast priors over rows
    return np.argmax(scores, axis=1)

pred = predict_nb(X, log_priors, log_lik)
print('pred', pred)
acc = float(np.mean(pred == labels))
print('train_acc', acc)
check('acc_reasonable', acc >= 0.8)

pred [1 1 1 0 0 0]
train_acc 1.0
OK: acc_reasonable


## Section 4 — Error Analysis

### Task 4.1: Most influential tokens per class
Show top tokens by log-likelihood ratio between classes.

# HINT: sort (log_lik[spam] - log_lik[ham])


In [10]:
# TODO
inv_vocab = {i:w for w,i in vocab.items()}
score = log_lik[1] - log_lik[0]
top = np.argsort(-score)[:5]
print('top spam tokens:', [inv_vocab[i] for i in top])

top spam tokens: ['win', 'money', 'free', 'now', 'claim']


---
## Submission Checklist
- All TODOs completed
- Train accuracy shown
- Top tokens printed
- Checkpoint questions answered
